In [1]:
"""Test pre-trained RGB model on a single video.

Date: 01/15/18
Authors: Bolei Zhou and Alex Andonian

This script accepts an mp4 video as the command line argument --video_file
and averages ResNet50 (trained on Moments) predictions on num_segment equally
spaced frames (extracted using ffmpeg).

Alternatively, one may instead provide the path to a directory containing
video frames saved as jpgs, which are sorted and forwarded through the model.

ResNet50 trained on Moments is used to predict the action for each frame,
and these class probabilities are average to produce a video-level predction.

Optionally, one can generate a new video --rendered_output from the frames
used to make the prediction with the predicted category in the top-left corner.

"""

import os
import argparse
import moviepy.editor as mpy
import time

import torch.optim
import torch.nn.parallel
from torch.nn import functional as F
import sys
sys.path.append("../moments_models/")

from prj_utils import *
import pygame

import numpy as np 
import cv2


import models

#from utils import extract_frames, load_frames, render_frames


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import moviepy.editor as mpe
video = mpe.VideoFileClip('mrbean.mp4')
#np_frame = video.get_frame(2)
#np_frame.ipython_display()
# n_frames = sum(1 for x in video.iter_frames())
# print(n_frames)
len=0

# images=[]
# for frame in video.iter_frames():
#     new_im = Image.fromarray(frame)
#     images.append(new_im)
#     display(new_im)
#     len=len+1
#     if len>100:
#         break;




In [42]:
framenumbers = range(50,75)
frames = [Image.fromarray(video.get_frame(i)).convert('RGB') for i in framenumbers]


In [43]:
# Load model

arch = 'resnet3d50'
model = models.load_model(arch)
transform = models.load_transform()

# Get dataset categories
categories = models.load_categories()
eat_idx = categories.index("eating")

In [44]:
# video_file = "mrbean.mp4"

# # Load the video frame transform
# transform = models.load_transform()

# frames = extract_frames(video_file, 50)

# # [num_frames, 3, 224, 224]
t = transform(frames[0])
t.shape

torch.Size([3, 224, 224])

In [45]:
if arch == 'resnet3d50':
    # [1, num_frames, 3, 224, 224]
    inp = torch.stack([transform(frame) for frame in frames], 1).unsqueeze(0)
else:
    # [num_frames, 3, 224, 224]
    inp = torch.stack([transform(frame) for frame in frames])



torch.Size([1, 3, 25, 224, 224])

In [46]:
def predict(inp,models):
    # Make video prediction
    start = time.time()
    with torch.no_grad():
        logits = model(inp)
        probs = F.softmax(logits, 1)
        end = time.time()
        return probs, (end - start)

In [47]:
probs,total = predict(inp,models)
print(f'Input size: {inp.shape}, time for prediction: {total} sec')

# Convert from Tensor to list
eat_probs = probs[:,eat_idx].tolist()


Input size: torch.Size([1, 3, 25, 224, 224]), time for prediction: 7.458142280578613 sec


In [41]:
probs[:,eat_idx]

tensor([0.1267])

In [36]:

# # Output the prediction.
# video_name = args.frame_folder if args.frame_folder is not None else args.video_file
# print('RESULT ON ' + video_name)
# for i in range(0, 5):
#     print('{:.3f} -> {}'.format(probs[i], categories[idx[i]]))

# Render output frames with prediction text.

prediction = [str(round(i,5)) for i in eat_probs]
rendered_frames = render_frames_2(frames, prediction)
clip = mpy.ImageSequenceClip(rendered_frames, fps=1)
#animation = mpy.ipython_display(clip)
clip.ipython_display(fps=10, loop=1, autoplay=1)


IndexError: list index out of range